# Explore evaluation results

### Arguments

In [ ]:
# Paths
model_path = '../runs/ws_post/model_finished.pth'
data_path = '../data/ws_cropped/'

# Parameters
device = 'cuda'
batch_size = 1
workers = 4
draw_threshold = 0.5
DPI = 220
vert_size = 500
line_width = 2
draw_label = True

### Code

In [ ]:
# Notebooks are stored in 'notebooks/' which breaks my imports
import sys
sys.path.insert(0, '..')

import time
from coco_utils import get_coco  # get_coco_kp
from torchvision import transforms
import torchvision
import models.detection
import transforms as T
import torch
import utils
from matplotlib.pyplot import figure, imshow, show
import matplotlib
import numpy as np

convert_to_pil = torchvision.transforms.ToPILImage()

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

# Datasets
dataset_train, num_classes, label_names = get_coco(data_path, image_set='train')
dataset_test, _, _ = get_coco(data_path, image_set='val')

# Samplers
train_sampler = torch.utils.data.SequentialSampler(dataset_train)
test_sampler = torch.utils.data.SequentialSampler(dataset_test)

train_batch_sampler = torch.utils.data.BatchSampler(
    train_sampler, batch_size, drop_last=True)

# Loaders
data_loader_train = torch.utils.data.DataLoader(
    dataset_train, batch_sampler=train_batch_sampler, num_workers=workers,
    collate_fn=utils.collate_fn)
data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1,
    sampler=test_sampler, num_workers=workers,
    collate_fn=utils.collate_fn)

# Load checkpoint
checkpoint = torch.load(model_path, map_location=device)
label_names = checkpoint['label_names']

# Set up model
model = models.detection.fasterrcnn_resnet50_fpn(
    num_classes=len(label_names) + 1, pretrained_backbone=False
)
model.to(device)
model.load_state_dict(checkpoint['model'])
model.eval()
print('Done loading model')

In [ ]:
matplotlib.rcParams['figure.dpi'] = DPI

def print_inference_results(data_loader, model):
    images_evaluated = 0
    for image, targets in data_loader:
        pre_model_image = image[0]

        image = list(img.to(device) for img in image)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # torch.cuda.synchronize()
        model_time = time.time()
        outputs = model(image)

        outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time


        scores = outputs[0]['scores']
        top_scores_filter = scores > draw_threshold
        top_scores = scores[top_scores_filter]
        top_boxes = outputs[0]['boxes'][top_scores_filter]
        top_labels = outputs[0]['labels'][top_scores_filter]
        image_with_boxes = utils.draw_boxes(
            pre_model_image, top_boxes, top_labels, label_names, scores,
            vert_size=vert_size, line_width=line_width, draw_label=draw_label
        )
        print(f"# {images_evaluated}")
        print(label_names[top_labels - 1] if len(top_labels) > 1 else [label_names[top_labels - 1]])
        figure()
        imshow(np.asarray(convert_to_pil(image_with_boxes)))
        show()
        images_evaluated += 1


In [ ]:
# Run data_loader_train or data_loader_test, but not both together or you will probably run out of GPU memory
print_inference_results(data_loader_test, model)